In [1]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size=14)
@njit
def rolling_mean_30(x):
    return rolling_mean(x, window_size=30)

In [3]:
def format_df_to_mlforecast(df, date_col, target_col, unique_id='mean'):
    df_ = df.rename({
        date_col: "ds",
        # target_col: 'y',
    }, axis=1)

    df_['ds'] = pd.to_datetime(df_['ds'])

    df_['y'] = df_[target_col].copy()
    # df_.drop(columns=target_col)

    df_['unique_id'] = unique_id
    return df_

In [4]:
selected_sensors_df = pd.read_csv("../data/selected_sensors2_cleaned.csv", index_col=0)

In [5]:
scenarios_sensors = {
    # 0: 1, 4372603
    # "0_12M_train_7M_test": {"train_start": "2017-03-25", "train_end": "2018-03-25", "test_start": "2018-03-26", "test_end": "2018-10-10"},
    '2': {
        "26M_train":  {"train_start": "2017-04-01", "train_end": "2019-06-01"},
        "24M_train":  {"train_start": "2017-04-01", "train_end": "2019-04-01"},
        "22M_train":  {"train_start": "2017-04-01", "train_end": "2019-02-01"},
        "20M_train":  {"train_start": "2017-04-01", "train_end": "2018-12-01"},
        "18M_train":  {"train_start": "2017-04-01", "train_end": "2018-10-01"},
        "12M_train":  {"train_start": "2017-04-01", "train_end": "2018-04-01"},
        "10M_train":  {"train_start": "2017-04-01", "train_end": "2018-01-25"},
        # "8M_train":   {"train_start": "2017-04-01", "train_end": "2017-10-25"},
        
        # Non-Heating Periods
        # "NH_3M_train":  {"train_start": "2017-04-15", "train_end": "2017-07-15"},
        # "NH_4M_train":  {"train_start": "2017-04-15", "train_end": "2017-08-15"},
        # "NH_2M_train":  {"train_start": "2017-04-15", "train_end": "2017-06-15"},
        # "NH_1M_train":  {"train_start": "2017-04-15", "train_end": "2017-05-15"},
        # "NH_15D_train": {"train_start": "2017-04-15", "train_end": "2017-04-30"},
        # "NH_feb_2M_train": {"train_start": "2017-02-15", "train_end": "2017-04-15"},
        # "NH_feb_1M_train": {"train_start": "2017-02-15", "train_end": "2017-04-15"},
        # "NH_mar_2M_train": {"train_start": "2017-03-15", "train_end": "2017-05-15"},
        # "NH_mar_1M_train": {"train_start": "2017-03-15", "train_end": "2017-04-15"},

        # # Heating Periods
        # "H_5M_train":     {"train_start": "2017-06-01", "train_end": "2017-11-01"},
        # "H_3M_jul_train": {"train_start": "2017-07-01", "train_end": "2017-10-10"},
        # "H_3M_sep_train": {"train_start": "2017-09-01", "train_end": "2017-12-10"},
        # "H_3M_nov_train": {"train_start": "2017-11-01", "train_end": "2018-02-10"},
        },
}
# scenarios_sensors['5'] = scenarios_sensors['2'].copy()
# scenarios_sensors['6'] = scenarios_sensors['2'].copy()

In [6]:
from MLForecastPipeline import *

In [7]:
def split_data(df, scenario, date_col="ds"):
    """Extracts train and test data based on train end date."""
    train_data = df[df[date_col] <= scenario['train_end']]
    test_start = pd.to_datetime(scenario['train_end']) + pd.Timedelta(days=1)
    test_data = df[df[date_col] >= test_start]
    return train_data, test_data

In [8]:
import warnings
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.special import boxcox1p, inv_boxcox1p

class TimeSeriesPreprocessor:
    def __init__(self, seasonal_period=365):
        """
        Preprocessor for time series data with optional transformations.

        Args:
        - seasonal_period (int): Periodicity of the seasonal component (e.g., 365 for yearly seasonality).
        """
        self.seasonal_period = seasonal_period
        self.trend = None
        self.seasonality = None
        self.lambda_param = None
        self.mean = None
        self.std = None
        self.lambdas = None
        self.original_first_value = None
        
    def remove_trend(self, series, method='diff'):
        """Removes trend using differencing (default) or decomposition (if enough data)."""
        try:
            if method == 'decomposition':
                if len(series) < (2 * self.seasonal_period):  # Ensure enough data
                    warnings.warn(f"Skipping decomposition: Data is too short (min required: {2*self.seasonal_period}). Falling back to differencing.")
                    method = 'diff'  # Fallback to differencing
            
                else:
                    decomposition = seasonal_decompose(series, period=self.seasonal_period)
                    self.trend = decomposition.trend
                    return series - decomposition.trend
            
            if method == 'diff':  # Default method
                self.original_first_value = series[0]
                return np.diff(series, prepend=series[0])
        
        except Exception as e:
            warnings.warn(f"Failed to remove trend: {e}")
            return series

    def remove_seasonality(self, series):
        """Removes seasonality only if data is long enough to support it."""
        if len(series) < (2 * self.seasonal_period):  # Ensure enough data
            warnings.warn("Skipping seasonality removal: Data is too short for a full seasonal cycle.")
            return series
        
        try:
            decomposition = seasonal_decompose(series, period=self.seasonal_period)
            self.seasonality = decomposition.seasonal
            return series - decomposition.seasonal
        except Exception as e:
            warnings.warn(f"Failed to remove seasonality: {e}")
            return series

    def apply_boxcox(self, series, lambda_param=None):
        """Applies Box-Cox transformation (handles errors)."""
        try:
            if lambda_param is None:
                self.lambda_param = stats.boxcox_normmax(series + 1)
            else:
                self.lambda_param = lambda_param
            return boxcox1p(series, self.lambda_param), self.lambda_param
        except Exception as e:
            warnings.warn(f"Failed to apply Box-Cox transformation: {e}")
            return series, -1

    def apply_log(self, series):
        """Applies log transformation (skips if negative values exist)."""
        try:
            if (series <= 0).any():
                warnings.warn("Skipping log transformation (negative values detected).")
                return series
            return np.log(series)
        except Exception as e:
            warnings.warn(f"Failed to apply log transformation: {e}")
            return series

    def standardize(self, series):
        """Applies standardization (z-score normalization)."""
        try:
            self.mean = series.mean()
            self.std = series.std()
            return (series - self.mean) / self.std
        except Exception as e:
            warnings.warn(f"Failed to standardize series: {e}")
            return series

    def fit_transform(self, series, steps):
        """
        Dynamically applies transformations based on provided steps.
        
        Args:
        - series (np.array or pd.Series): Input time series data.
        - steps (list): List of transformations to apply (e.g., ['trend', 'seasonality', 'boxcox', 'log', 'standardize']).
        
        Returns:
        - Transformed series (np.array).
        """
        transformed = series.copy()

        for step in steps:
            if step == 'trend':
                transformed = self.remove_trend(transformed)
            elif step == 'seasonality':
                transformed = self.remove_seasonality(transformed)  # Only if data is long enough
            elif step == 'boxcox':
                transformed, _ = self.apply_boxcox(transformed)
            elif step == 'log':
                transformed = self.apply_log(transformed)
            elif step == 'standardize':
                transformed = self.standardize(transformed)
        
        return transformed

    def inverse_transform_predictions(self, predictions):
        """
        Inverses transformations in the correct order.
        
        Args:
        - predictions (np.array): Transformed predictions.
        
        Returns:
        - Original scale predictions.
        """
        transformed = predictions.copy()

        if 'standardize' in self.transforms:
            transformed = (transformed * self.std) + self.mean

        if 'log' in self.transforms:
            transformed = np.exp(transformed)

        if 'boxcox' in self.transforms:
            if self.lambda_param != -1:
                transformed = inv_boxcox1p(transformed, self.lambda_param)

        if 'seasonality' in self.transforms and self.seasonality is not None:
            start_idx = len(self.trend) % self.seasonal_period if self.trend is not None else 0
            seasonal_indices = np.arange(start_idx, start_idx + len(predictions)) % self.seasonal_period
            seasonal_pattern = self.seasonality[seasonal_indices]
            transformed = transformed + seasonal_pattern

        if 'trend' in self.transforms:
            if hasattr(self, 'original_first_value'):
                reversed_series = np.cumsum(transformed)
                transformed = reversed_series + self.original_first_value
            else:
                warnings.warn("Trend removal used differencing, but original_first_value is not set. Skipping trend reversal.")
        
        return transformed

    def create_pipeline(self, series, steps):
        """
        Generates all valid transformation pipelines.

        Args:
        - series (np.array): Time series data.
        - steps (list): List of possible transformations.

        Returns:
        - dict with transformed series for each pipeline.
        """
        import itertools

        results = {'original': series.copy()}
        self.lambdas = {}

        for r in range(1, len(steps) + 1):
            for combo in itertools.combinations(steps, r):
                transformed = series.copy()
                name = []

                for step in combo:
                    if step == 'trend':
                        transformed = self.remove_trend(transformed)
                        name.append('trend')
                    elif step == 'seasonality':
                        transformed = self.remove_seasonality(transformed)
                        name.append('seasonality')
                    elif step == 'boxcox':
                        transformed, lambda_param = self.apply_boxcox(transformed)
                        self.lambdas["_".join(combo)] = lambda_param
                        name.append('boxcox')
                    elif step == 'log':
                        transformed = self.apply_log(transformed)
                        name.append('log')
                    elif step == 'standardize':
                        transformed = self.standardize(transformed)
                        name.append('standardize')

                    transformed = transformed[~np.isnan(transformed)]

                results['_'.join(name)] = transformed
                
        return results


In [9]:
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA, AutoETS, AutoCES, AutoTheta, CrostonClassic, CrostonSBA,
    AutoTBATS, SeasonalExponentialSmoothingOptimized, HoltWinters, Holt,
    SeasonalWindowAverage, ADIDA, CrostonOptimized, IMAPA, TSB,
    MSTL, MFLES, OptimizedTheta, DynamicOptimizedTheta, GARCH
)
models_dict = {
    "MSTL_abc_ARIMA": MSTL(season_length=[7, 30, 365], trend_forecaster=AutoARIMA()),
    "MSTL_ac_ARIMA": MSTL(season_length=[7, 365], trend_forecaster=AutoARIMA()),
    "MSTL_a_ARIMA": MSTL(season_length=[7], trend_forecaster=AutoARIMA()),
    "MSTL_ac_MFLES1": MSTL(season_length=[7, 365], trend_forecaster=MFLES(season_length=1)),
    "MSTL_ac_MFLES2": MSTL(season_length=[7, 365], trend_forecaster=MFLES(season_length=1, fourier_order=3)),
    "MSTL_ac_MFLES3": MSTL(season_length=[7, 365], trend_forecaster=MFLES(season_length=1, fourier_order=5)),
}

In [14]:
import numpy as np
import pandas as pd
import hashlib
import joblib
import json
import os
from copy import deepcopy
from statsforecast import StatsForecast

RUN_NUM = "run_9"

# Ensure save directory exists
save_dir = f"results/{RUN_NUM}"
os.makedirs(save_dir, exist_ok=True)

passed_models_info = {}

def mstl_name_mapper(model_name):
    if 'MSTL' in model_name:
        return "MSTL"
    return model_name

# Step 1: Preprocess and Generate Transformed Versions
for sensor_id, train_scenarios in scenarios_sensors.items():
    sensor_id = str(sensor_id)
    
    for train_label, scenario in train_scenarios.items():
        print(f"Processing Sensor {sensor_id}, Scenario: {train_label}")

        # Load the dataset for the sensor
        formatted_df = format_df_to_mlforecast(
            selected_sensors_df[['full_date', sensor_id]], 'full_date', sensor_id, unique_id=sensor_id
        )
        formatted_df = formatted_df[['ds', 'y', 'unique_id']]

        # Split the dataset using predefined scenario
        train_df, test_df = split_data(formatted_df, scenario)

        # Preprocess data with all transformations
        data = train_df['y'].values.copy()
        preprocessor = TimeSeriesPreprocessor(seasonal_period=365)
        steps = ['log', 'trend', 'seasonality', 'boxcox', 'standardize']
        transformed_series = preprocessor.create_pipeline(data, steps)

        # Step 2: Train All Models
        for name, series in transformed_series.items():
            print(f"Training on transformed version: {name}")

            # Prepare scenario-based training and test sets
            train_df['y'] = series
            test_series = test_df['y'].values.copy()

            if len(series) < 30:
                continue

            results = {}
            for model_name, model in models_dict.items():
                try:
                    sf = StatsForecast(models=[model], freq='D', n_jobs=-1)
                    sf.fit(train_df)
                    forecasts = sf.predict(h=len(test_series)).reset_index()
                    # Get predictions & apply inverse transformations before computing metrics
                    target_col = list(set(forecasts.columns) - set(['ds', 'unique_id']))[0]
                    transformed_forecast = forecasts[target_col].values

                    # Apply inverse transformations
                    distr_preprocessor = deepcopy(preprocessor)
                    distr_preprocessor.transforms = name.split('_')
                    if "boxcox" in name:
                        distr_preprocessor.lambda_param = preprocessor.lambdas[name]
                    
                    orig_preds = distr_preprocessor.inverse_transform_predictions(transformed_forecast)

                    # Compute metrics **AFTER inverse transformation**
                    test_df_copy = test_df.copy()
                    test_df_copy['forecast'] = orig_preds

                    # test_df_copy.plot(x='ds', y=['y', 'forecast'], title=f"Forecast vs Actuals for {model_name} with {name}")
                    # plt.xlabel('Date')
                    # plt.ylabel('Value')
                    # plt.show()

                    error_dict = {}
                    for test_length in [30, 60, 90, 120, 150, 180, 240, 300, 360, 480, 600, 720, len(test_series)]:
                        eval_subset = test_df_copy.iloc[:test_length]
                        error_dict[f"test_{test_length}_days"] = mape_met(eval_subset['y'].values, eval_subset['forecast'].values)

                    results[model_name] = {
                        "MAPE": error_dict[f"test_{len(test_series)}_days"],
                        # "RMSE": rmse_met(test_df_copy['y'].values, test_df_copy['forecast'].values),
                        "Predictions": orig_preds,
                        "Errors": error_dict,
                    }

                except Exception as e:
                    print(f"Skipping {model_name} on {name} due to error: {e}")
                    continue

            # Select best model based on lowest final horizon MAPE
            try:
                best_model = min(results, key=lambda x: results[x]['MAPE'])
                print(f"Best Model for {name}: {best_model} (MAPE: {results[best_model]['MAPE']:.2f}%)")
            except Exception as e:
                print("some error:", e)

            # Step 3: Store and Save Models
            if results[best_model]['MAPE'] < 50:
                model_hash = hashlib.md5(f"{sensor_id}_{train_label}_{name}_{best_model}".encode()).hexdigest()[:8]
                unique_model_id = f"{sensor_id}_{train_label}_{name}_{best_model}_{model_hash}"

                print(f"Saving model {unique_model_id}")

                model_filename = f"{save_dir}/{unique_model_id}.joblib"
                joblib.dump(models_dict[best_model], model_filename)

                passed_models_info[unique_model_id] = {
                    "sensor": sensor_id,
                    "train_label": train_label,
                    "transformation": name,
                    "model_name": best_model,
                    "MAPE": results[best_model]['MAPE'],
                    "Errors": results[best_model]['Errors'],
                    "file_path": model_filename,
                }

# Step 4: Save Metadata
with open(f"{save_dir}/passed_models_metadata.json", "w") as f:
    json.dump(passed_models_info, f, indent=4)

pd.DataFrame.from_dict(passed_models_info, orient="index").to_csv(f"{save_dir}/passed_models_metadata.csv")

print("All models and metadata saved successfully.")


Processing Sensor 2, Scenario: 26M_train


C:\Users\PC314\AppData\Local\Temp\ipykernel_17148\2310296225.py:69: UserWarning: Failed to apply Box-Cox transformation: The `x` argument of `boxcox_normmax` must contain only positive, finite, real numbers.
  warnings.warn(f"Failed to apply Box-Cox transformation: {e}")


Training on transformed version: original


C:\Users\PC314\AppData\Local\Temp\ipykernel_17148\3122320333.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['y'] = series
c:\Users\PC314\anaconda3\Lib\site-packages\statsforecast\core.py:494: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\PC314\anaconda3\Lib\site-packages\statsforecast\core.py:494: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\PC314\anaconda3\Lib\site-packages\statsforecast\co

Best Model for original: MSTL_ac_MFLES2 (MAPE: 38.07%)
Saving model 2_26M_train_original_MSTL_ac_MFLES2_8e5ff900
Training on transformed version: log


...

c:\Users\PC314\anaconda3\Lib\site-packages\statsforecast\core.py:494: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\PC314\anaconda3\Lib\site-packages\statsforecast\mfles.py:301: RuntimeWarning: overflow encountered in scalar multiply
  slope = (M * x_y_sum - x_sum * y_sum) / (M * x_sq_sum - x_sum**2)


Skipping MSTL_ac_MFLES1 on log_trend_seasonality_boxcox_standardize due to error: could not broadcast input array from shape (930,) into shape (986,)


c:\Users\PC314\anaconda3\Lib\site-packages\statsforecast\mfles.py:301: RuntimeWarning: overflow encountered in scalar multiply
  slope = (M * x_y_sum - x_sum * y_sum) / (M * x_sq_sum - x_sum**2)


Skipping MSTL_ac_MFLES2 on log_trend_seasonality_boxcox_standardize due to error: could not broadcast input array from shape (930,) into shape (986,)
Skipping MSTL_ac_MFLES3 on log_trend_seasonality_boxcox_standardize due to error: could not broadcast input array from shape (930,) into shape (986,)
Best Model for log_trend_seasonality_boxcox_standardize: MSTL_a_ARIMA (MAPE: 1661.88%)
All models and metadata saved successfully.


c:\Users\PC314\anaconda3\Lib\site-packages\statsforecast\mfles.py:301: RuntimeWarning: overflow encountered in scalar multiply
  slope = (M * x_y_sum - x_sum * y_sum) / (M * x_sq_sum - x_sum**2)
